In [37]:
import pandas as pd
import numpy as np

In [38]:
# IO path variables

INPUT_FILE_PATH = "FY26 Dec Budget Control Report.xlsx"
OUTPUT_FILE_PATH = "FY26 Dec Approval.xlsx"


In [39]:
# Read file in

df_raw = pd.read_excel(INPUT_FILE_PATH, skipfooter=1)

## Decide to approve or hold based on individual publication

In [40]:
df_raw.columns

Index(['MEDIA', 'CLICODE', 'CLINAME', 'PRDCODE', 'PRDNAME', 'ESTNUM',
       'ESTNAME', 'MPA', 'CAMPAIGN', 'BLBDATE', 'PUBLICATION', 'STATION',
       'NET ORDERED', 'NET PAID', 'NET PAYABLE', 'NET BILLED', 'NET BILLABLE',
       'COMMISSIO0%', 'PREP', 'SREP', 'CONTACT', 'RECD', 'CID', 'INV NUM',
       'I/O', 'Commission', 'MISSING CONTACT', 'POT. COMMISSION ADD',
       'POT. COMMISSION REMOVE', 'COMMISSION ($)', 'COMMISSIONABLE',
       'SERVICE YEAR', 'SERVICE MONTH', 'fMEDIA', 'nMPA', 'LANGUAGE', 'LOB',
       'ORDERED (w/ Commission + Tax)', 'BILLED (w/ Commission + Tax)',
       'BILLABLE (w/ Commission + Tax)', 'TOTAL SPEND (w/ Commission + Tax)',
       'MPA TOTAL (w/ Commission + Tax)', 'Comment'],
      dtype='object')

In [42]:
rearrange_col= ['LOB', 'MPA', 'CAMPAIGN', 'PUBLICATION', 'ESTNUM', 'BLBDATE', 'NET ORDERED', 'NET PAID', 'NET PAYABLE', 'NET BILLED', 'NET BILLABLE',
       'RECD', 'CID', 'INV NUM',
       'I/O', 'SERVICE YEAR', 'SERVICE MONTH', 'MEDIA', 'PRDCODE']

df = df_raw[rearrange_col]

In [43]:
# Filter by Media and remove NETWORK TV, Radio, Search

df = df[df['MEDIA'].apply(lambda x: x not in ['NETWORK TV', 'SEARCH', 'RADIO'])]


In [44]:
# Format the dates

df['MOS'] = pd.to_datetime("20" + df['BLBDATE'].astype('str').str[:2] + "-" + df['BLBDATE'].astype('str').str[2:4] + "-01")

# Rearrange columns and drop BLBDATE column
rearrange_col= ['LOB', 'MPA', 'CAMPAIGN', 'PUBLICATION', 'ESTNUM', 'MOS', 'NET ORDERED', 'NET PAID', 'NET PAYABLE', 'NET BILLED', 'NET BILLABLE',
       'RECD', 'CID', 'INV NUM',
       'I/O', 'SERVICE YEAR', 'SERVICE MONTH', 'MEDIA', 'PRDCODE']
df = df[rearrange_col]
# TV has different BLBDATE format and creates error


In [45]:
# Filter by LOB

# Westjet doesn't appear in LOB, it's under Cards LOB. Need PRD = WSJ

df = df[df['LOB'].apply(lambda x: x in ['Caribbean', 'Insurance', 'Deposits', 'Suncor', 'Cultural Media']) | (df['PRDCODE'] == 'WSJ') | (df['PRDCODE'] == 'QMV') | (df['PRDCODE'] == 'EBA')]



df.sort_values(by=['LOB', 'MPA', 'CAMPAIGN', 'PUBLICATION', 'ESTNUM', 'MOS'], inplace=True)

In [46]:
# Drop some columns and move Media column
new_cols = ['LOB', 'MPA', 'CAMPAIGN','MEDIA', 'PUBLICATION', 'ESTNUM', 'MOS', 'NET ORDERED',
       'NET PAID', 'NET PAYABLE', 'NET BILLED', 'NET BILLABLE', 'RECD']

df = df[new_cols]

In [47]:
# Define a function to code Reconciled column to enable numeric operations
# Yes = 0, No = 1

def no_to_one(x):
    if x == 'YES':
        return 0
    else:
        return 1

In [48]:
# verification
# df['RECD'].to_frame().join( df['RECD'].apply(no_to_one).to_frame(), lsuffix='_s1', rsuffix='_s2').to_excel("recd.xlsx")

# Add the numeric RECON column to df
df['RECD Numeric'] = df['RECD'].apply(no_to_one)

### Group all the media buys by MPA, Campaign, and Publication

In [51]:
publications = df.groupby(['MPA', 'CAMPAIGN', 'PUBLICATION'])
len(publications)

73

In [52]:
# Filter for rows with negative sum of Net Billable
neg_df = publications.filter(lambda x: x['NET BILLABLE'].sum() < 0)
pos_df = publications.filter(lambda x: x['NET BILLABLE'].sum() >= 0)

In [55]:
# Split neg_df into 2 dataframes: not_all_reconciled and all_reconciled

In [58]:
# Divide neg_df into 2

not_all_reconciled_df = publications_recd.filter(lambda x: x['RECD Numeric'].mean() > 0)
all_reconciled_df = publications_recd.filter(lambda x: x['RECD Numeric'].mean() == 0)

In [61]:
# Approve 1. completely reconciled media buys
#         2. media buys whose 'Net Billable' is positive
# Yes = 0, No = 1

not_all_reconciled_df['Approve'] = 1
all_reconciled_df['Approve'] = 0
pos_df['Approve'] = 0

In [63]:
# Put all media buys together into one dataframe

original_no_df = pd.concat([not_all_reconciled_df, all_reconciled_df, pos_df], ignore_index=True)

## Propagate the result to other lines with same MOS and ESTNUM
### :Hold all other lines that share the MOS and ESTNUM with lines that are held above

In [66]:
publications2 = original_no_df.groupby(['MPA', 'CAMPAIGN', 'ESTNUM', 'MOS'])
len(publications2)

163

In [69]:
# Filter for rows to approve

approve_df = publications2.filter(lambda x: x['Approve'].mean() == 0)
hold_df = publications2.filter(lambda x: x['Approve'].mean() > 0)

In [70]:
# Record final approval status

hold_df['Final Approve'] = 1
approve_df['Final Approve'] = 0

In [ ]:
final_df = pd.concat([hold_df, approve_df], ignore_index=True)
final_df.to_excel(OUTPUT_FILE_PATH)